In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
%pip install git+https://github.com/databrickslabs/dbignite.git

In [0]:
%restart_python

In [0]:
# Databricks Secret Scope for the Run-As User
dbutils.widgets.text(
  "databricks_secret_scope"
  ,spark.sql("select current_user()").collect()[0][0].split(sep="@")[0].replace(".", "-")
  ,"User's Personal DB Secrets Scope"
)

# Databricks Secret Key for the GitHub PAT in the User's Scope
dbutils.widgets.text(
  "github_token_secret_key"
  ,"gh_pat"
  ,"GitHub Token DB Secret Key"
)

# Run-As User's GitHub User Name
dbutils.widgets.text(
  "github_username"
  ,spark.sql("select current_user()").collect()[0][0]
  ,"User's Personal GitHub Username"
)

In [0]:

from urllib.parse import quote

github_username = quote(dbutils.widgets.get("github_username"))
github_token = quote(dbutils.secrets.get(scope = dbutils.widgets.get("databricks_secret_scope"), key = dbutils.widgets.get("github_token_secret_key")))

print(f"""
   github_username: {github_username}
   github_token: {github_token}   
""")

## Connect to Redox

In [0]:
redox_private_key = dbutils.secrets.get(scope = dbutils.widgets.get("databricks_secret_scope"), key = "redox_private_key")
redox_client_id = dbutils.secrets.get(scope = dbutils.widgets.get("databricks_secret_scope"), key = "redox_client_id")
# redox_source_id = dbutils.secrets.get(scope = dbutils.widgets.get("databricks_secret_scope"), key = "redox_source_id")
redox_source_id = "91b0ab2f-7b86-441d-9cbf-9b9a6d648d59"
redox_public_kid = dbutils.secrets.get(scope = dbutils.widgets.get("databricks_secret_scope"), key = "redox_kid")

print(f""" 
      redox_private_key: {redox_private_key}
      redox_client_id: {redox_client_id}
      redox_source_id: {redox_source_id}
      redox_public_kid: {redox_public_kid}
""")

In [0]:
import json

redox_auth_json = f"""
{{
  "kty": "RSA",
  "kid": "{redox_public_kid}",
  "alg": "RS384",
  "use": "sig"
}}
"""

json.loads(redox_auth_json)

In [0]:
from redoxwrite.auth import * 
from redoxwrite.endpoint import *

auth = RedoxApiAuth(
  redox_client_id
  ,redox_private_key
  ,redox_auth_json
  ,redox_source_id
)
print("Is connection successful? " + str(auth.can_connect()))

In [0]:
#All Redox FHIR request URLs start with this base: https://api.redoxengine.com/fhir/R4/[organization-name]/[environment-type]/
redox_base_url = 'https://api.redoxengine.com/fhir/R4/redox-fhir-sandbox/Development/'

rapi = RedoxApiRequest(auth, base_url = redox_base_url)

In [0]:
result = rapi.make_request("post", resource="Patient", action="_search", data = "{'identifier' : 'urn:redox:redox-fhir-sandbox:MR|kyHGADnvX3xbkU4V9ayaqh'}")

In [0]:
if result['response']['response_status_code'] != 200:
  print("Failed to update the patient information")
print(json.dumps(json.loads(result['response']['response_text']), indent=2))

In [0]:
# approved_service = """
# {
#   "id": "RedoxClaimResponseBundleComprehensiveExample",
#   "type": "collection",
#   "entry": [
#     {
#       "resource": {
#         "id": "RedoxPAClaimResponseExample",
#         "use": "preauthorization",
#         "item": [
#           {
#             "extension": [
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-administrationReferenceNumber",
#                 "valueString": "REF456"
#               },
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemPreAuthIssueDate",
#                 "valueDate": "2021-08-23"
#               },
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemPreAuthPeriod",
#                 "valuePeriod": {
#                   "end": "2021-09-23",
#                   "start": "2021-08-23"
#                 }
#               },
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemTraceNumber",
#                 "valueIdentifier": {
#                   "value": "23462346"
#                 }
#               },
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemAuthorizedDetail",
#                 "extension": [
#                   {
#                     "url": "productOrServiceCode",
#                     "valueCodeableConcept": {
#                       "text": "Right heart catheterization",
#                       "coding": [
#                         {
#                           "code": "93451",
#                           "system": "http://www.ama-assn.org/go/cpt"
#                         }
#                       ]
#                     }
#                   },
#                   {
#                     "url": "quantity",
#                     "valueQuantity": {
#                       "value": 1
#                     }
#                   },
#                   {
#                     "url": "unitPrice",
#                     "valueMoney": {
#                       "value": 4966,
#                       "currency": "USD"
#                     }
#                   }
#                 ]
#               }
#             ],
#             "adjudication": [
#               {
#                 "category": {
#                   "coding": [
#                     {
#                       "code": "submitted",
#                       "system": "http://terminology.hl7.org/CodeSystem/adjudication"
#                     }
#                   ]
#                 },
#                 "extension": [
#                   {
#                     "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-reviewAction",
#                     "extension": [
#                       {
#                         "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-reviewActionCode",
#                         "valueCodeableConcept": {
#                           "coding": [
#                             {
#                               "code": "A1",
#                               "system": "https://codesystem.x12.org/005010/306",
#                               "display": "Certified in total"
#                             }
#                           ]
#                         }
#                       },
#                       {
#                         "url": "number",
#                         "valueString": "AUTH1001"
#                       }
#                     ]
#                   }
#                 ]
#               }
#             ],
#             "itemSequence": 1
#           }
#         ],
#         "type": {
#           "coding": [
#             {
#               "code": "professional",
#               "system": "http://terminology.hl7.org/CodeSystem/claim-type"
#             }
#           ]
#         },
#         "status": "active",
#         "created": "2021-08-23T16:35:54.648Z",
#         "insurer": {
#           "reference": "Organization/RedoxPAInsurerOrganizationExample"
#         },
#         "outcome": "complete",
#         "patient": {
#           "reference": "Patient/RedoxPABeneficiaryExample"
#         },
#         "request": {
#           "reference": "Claim/RedoxPAServiceClaimExample",
#           "identifier": {
#             "type": {
#               "coding": [
#                 {
#                   "code": "ClaimID"
#                 }
#               ]
#             },
#             "value": "urn:uuid:97b8d6d0-ac3d-411a-b32a-e1d237aceb6a"
#           }
#         },
#         "requestor": {
#           "reference": "Organization/RedoxPARequestorOrganizationExample"
#         },
#         "processNote": [
#           {
#             "text": "Covered under extended benefits"
#           }
#         ],
#         "resourceType": "ClaimResponse"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPABeneficiaryExample",
#         "name": [
#           {
#             "given": [
#               "Timothy",
#               "Paul"
#             ],
#             "family": "Bixby"
#           }
#         ],
#         "gender": "male",
#         "birthDate": "2008-01-06",
#         "identifier": [
#           {
#             "value": "M23462346",
#             "system": "urn:redox:MRN"
#           }
#         ],
#         "resourceType": "Patient"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPACoverageWithRelatedSubscriberExample",
#         "class": [
#           {
#             "type": {
#               "coding": [
#                 {
#                   "code": "group",
#                   "system": "http://terminology.hl7.org/CodeSystem/coverage-class"
#                 }
#               ]
#             },
#             "value": "GRP2245"
#           }
#         ],
#         "payor": [
#           {
#             "reference": "Organization/RedoxPAInsurerOrganizationExample"
#           }
#         ],
#         "status": "active",
#         "subscriber": {
#           "reference": "RelatedPerson/RedoxPASubscriberExample"
#         },
#         "beneficiary": {
#           "reference": "Patient/RedoxPABeneficiaryExample"
#         },
#         "resourceType": "Coverage",
#         "subscriberId": "M2346623"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPARequestorOrganizationExample",
#         "name": "Good Health Clinics",
#         "active": true,
#         "address": [
#           {
#             "city": "Madison",
#             "line": [
#               "123 Main St."
#             ],
#             "state": "WI",
#             "country": "USA",
#             "district": "Dane",
#             "postalCode": "53703"
#           }
#         ],
#         "resourceType": "Organization"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPAInsurerOrganizationExample",
#         "name": "Aetna",
#         "active": true,
#         "address": [
#           {
#             "city": "Lexington",
#             "line": [
#               "PO Box 14080"
#             ],
#             "state": "KY",
#             "country": "USA",
#             "district": "Fayette",
#             "postalCode": "40512-4079"
#           }
#         ],
#         "resourceType": "Organization"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPAPractitionerRoleExamplePat",
#         "location": [
#           {
#             "display": "1st Street Family Clinic"
#           }
#         ],
#         "specialty": [
#           {
#             "coding": [
#               {
#                 "code": "394579002",
#                 "system": "http://snomed.info/sct",
#                 "display": "Cardiology"
#               }
#             ]
#           }
#         ],
#         "organization": {
#           "reference": "Organization/RedoxPARequestorOrganizationExample"
#         },
#         "practitioner": {
#           "reference": "Practitioner/RedoxPAPractitionerExamplePat"
#         },
#         "resourceType": "PractitionerRole"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPAPractitionerExamplePat",
#         "name": [
#           {
#             "given": [
#               "Pat"
#             ],
#             "family": "Granite"
#           }
#         ],
#         "address": [
#           {
#             "city": "Madison",
#             "line": [
#               "123 Main St."
#             ],
#             "state": "WI",
#             "country": "USA",
#             "district": "Dane",
#             "postalCode": "53703"
#           }
#         ],
#         "telecom": [
#           {
#             "value": "+16085551234"
#           }
#         ],
#         "identifier": [
#           {
#             "value": "4356789876",
#             "system": "http://hl7.org/fhir/sid/us-npi"
#           }
#         ],
#         "resourceType": "Practitioner"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPASubscriberExample",
#         "name": [
#           {
#             "given": [
#               "Barbara"
#             ],
#             "family": "Bixby"
#           }
#         ],
#         "patient": {
#           "reference": "RedoxPABeneficiary"
#         },
#         "relationship": [
#           {
#             "text": "Mother"
#           }
#         ],
#         "resourceType": "RelatedPerson"
#       }
#     },
#     {
#       "resource": {
#         "id": "RedoxPAServiceClaimExample",
#         "use": "preauthorization",
#         "item": [
#           {
#             "sequence": 1,
#             "extension": [
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-requestedService",
#                 "valueReference": {
#                   "reference": "ServiceRequest/RedoxPAServiceRequestExample"
#                 }
#               },
#               {
#                 "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemTraceNumber",
#                 "valueIdentifier": {
#                   "value": "23462346"
#                 }
#               }
#             ],
#             "productOrService": {
#               "text": "Coronary angiography with right heart catheterization",
#               "coding": [
#                 {
#                   "code": "93456",
#                   "system": "http://www.ama-assn.org/go/cpt"
#                 }
#               ]
#             }
#           }
#         ],
#         "type": {
#           "coding": [
#             {
#               "code": "professional",
#               "system": "http://terminology.hl7.org/CodeSystem/claim-type"
#             }
#           ]
#         },
#         "status": "active",
#         "created": "2021-09-02T18:44:13.307Z",
#         "enterer": {
#           "reference": "PractitionerRole/RedoxPAPractitionerRoleExamplePat"
#         },
#         "insurer": {
#           "reference": "Organization/RedoxPAInsurerOrganizationExample"
#         },
#         "patient": {
#           "reference": "Patient/RedoxPABeneficiaryExample"
#         },
#         "priority": {
#           "coding": [
#             {
#               "code": "normal",
#               "system": "http://terminology.hl7.org/CodeSystem/processpriority"
#             }
#           ]
#         },
#         "provider": {
#           "reference": "Organization/RedoxPARequestorOrganizationExample"
#         },
#         "diagnosis": [
#           {
#             "sequence": 1,
#             "diagnosisCodeableConcept": {
#               "coding": [
#                 {
#                   "code": "I42.9",
#                   "system": "http://hl7.org/fhir/sid/icd-10-cm",
#                   "display": "Cardiomyopathy, unspecified"
#                 }
#               ]
#             }
#           }
#         ],
#         "insurance": [
#           {
#             "focal": true,
#             "coverage": {
#               "reference": "Coverage/RedoxPACoverageExample"
#             },
#             "sequence": 1
#           }
#         ],
#         "identifier": [
#           {
#             "type": {
#               "coding": [
#                 {
#                   "code": "ClaimID"
#                 }
#               ]
#             },
#             "value": "urn:uuid:97b8d6d0-ac3d-411a-b32a-e1d237aceb6a"
#           }
#         ],
#         "resourceType": "Claim",
#         "supportingInfo": [
#           {
#             "category": {
#               "coding": [
#                 {
#                   "code": "additionalInformation",
#                   "system": "http://hl7.org/fhir/us/davinci-pas/CodeSystem/PASSupportingInfoType",
#                   "display": "Send additional paperwork or supporting information is sent for the request."
#                 }
#               ]
#             },
#             "sequence": 1,
#             "valueReference": {
#               "reference": "DocumentReference/RedoxPADocumentReferenceExample"
#             }
#           },
#           {
#             "category": {
#               "coding": [
#                 {
#                   "code": "freeFormMessage",
#                   "system": "http://hl7.org/fhir/us/davinci-pas/CodeSystem/PASSupportingInfoType",
#                   "display": "Written Confirmation."
#                 }
#               ]
#             },
#             "sequence": 2,
#             "valueString": "1"
#           }
#         ]
#       }
#     }
#   ],
#   "timestamp": "2021-10-21T19:09:02.452Z",
#   "identifier": {
#     "value": "urn:uuid:59f129cf-2eca-4965-88f2-452558af1934",
#     "system": "urn:ietf:rfc:3986"
#   },
#   "resourceType": "Bundle"
# }
# """

In [0]:
# result = rapi.make_request(http_method="post", resource="ClaimResponse", action="$respond", data=approved_service)

In [0]:
# result

In [0]:
# if result['response']['response_status_code'] != 200:
#   print("Failed to update the patient information")
# print(json.dumps(json.loads(result['response']['response_text']), indent=2))

In [0]:
enrollment_sdf = spark.table("mgiglia.hv_claims.enrollment")
display(enrollment_sdf)

In [0]:
from dbignite.writer.bundler import *
from dbignite.writer.fhir_encoder import *

In [0]:
help(Mapping)

In [0]:
maps = [
  Mapping('patient_id', 'Patient.id')
  ,Mapping('patient_gender', 'Patient.gender')
  ,Mapping('patient_year_of_birth', 'Patient.birthDate')
  ,Mapping('patient_zip3', 'Patient.address.postalCode')
  ,Mapping('patient_state', 'Patient.address.state')
  ,Mapping('date_start', 'Coverage.period.start')
  ,Mapping('date_end', 'Coverage.period.end')
  ,Mapping()
  ,Mapping('<url of a hardcoded system reference>', 'Patient.identifier.system', True)
	,Mapping('PATIENT_ID', 'Patient.id')]